Лабораторная работа №1. "Разработка классификатора методом k ближайших соседей"

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from google.colab import files


uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


1. Загрузить данные из файла heart.csv, используя функцию read_csv() библиотеки pandas

In [ ]:
data = pd.read_csv('diabetes.csv')

2. Вывести несколько строк датафрейма. Более подробная информация о датасете https://www.kaggle.com/mathchi/diabetes-data-set

In [ ]:
яdata

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


3. Проверить сбалансированность классов. Можно воспользоваться методом value_counts() датафрейма. Для каждого класса нужно вывести долю примеров от общего числа

In [ ]:
def balance_print_full(input):
  class_counts = input['Outcome'].value_counts()
  total_examples = len(input)
  for class_label, count in class_counts.items():
      class_fraction = count / total_examples
      print(f"Class {class_label}: {class_fraction:.3f}")
  print('')

def balance_print(input):
  class_counts = input.value_counts()
  total_examples = len(input)
  for class_label, count in class_counts.items():
      class_fraction = count / total_examples
      print(f"Class {class_label}: {class_fraction:.3f}")
  print('')

balance_print_full(data)

Class 0: 0.651
Class 1: 0.349



4. Провести процедуру нормирования признаков. Вариант 1: так, чтобы все признаки принимали значения в диапазоне $[0;1]$, используя преобразование $X^{*}=(X-X_{min})/{\Delta{X}}$ для каждого признака. Вариант 2: так, чтобы все признаки принимали значения в диапазоне $[-3\sigma;+3\sigma]$, используя преобразование $X^{*}=(X-\mu_{X})/{\sigma_{X}}$ для каждого признака.
Процедуру обезразмеривания оформить в виде функции

In [ ]:
def normalization(X):
  X_norm = (X - X.min()) / (X.max() - X.min())
  return X_norm

In [ ]:
X_norm = normalization(data.drop('Outcome', axis = 1))
Y = data.Outcome

In [ ]:
X_norm

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000
...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333


In [ ]:
X_norm.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,0.226180,0.607510,0.566438,0.207439,0.094326,0.476790,0.168179,0.204015
std,0.198210,0.160666,0.158654,0.161134,0.136222,0.117499,0.141473,0.196004
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.058824,0.497487,0.508197,0.000000,0.000000,0.406855,0.070773,0.050000
50%,0.176471,0.587940,0.590164,0.232323,0.036052,0.476900,0.125747,0.133333
75%,0.352941,0.704774,0.655738,0.323232,0.150414,0.545455,0.234095,0.333333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


5. Разделить выборку на три части: обучающую, валидационную и тестовую в соотношении 0.6:0.2:0.2. Выбор из исходной выборки должен осуществляться случайным образом. Можно воспользоваться функцией choice() библиотеки numpy.random. Можно также воспользоваться специальной функцией train_test_split() библиотеки sklearn.model_selection. Проверить сбалансированность классов в полученных выборках.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, Y_train, Y_temp = train_test_split(X_norm,Y, test_size = 0.4, random_state = 1)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size = 0.5, random_state = 1)

balance_print(Y_train)
balance_print(Y_val)
balance_print(Y_test)

Class 0: 0.654
Class 1: 0.346

Class 0: 0.617
Class 1: 0.383

Class 0: 0.675
Class 1: 0.325



6. Написать функцию knn_classification(X,y,X_new,k), реализующую метод k-ближайших соседей для задачи классификации. Параметрами функции являются: X,y - обучающая выборка, X_new - массив (или датафрейм) признаков для новых объектов, k - гиперпараметр метода (кол-во ближайших соседей, в функции предусмотреть возможность задания k по умолчанию). Функция должна возвращать прогноз для новых объектов в виде массива классов (y_pred).

In [ ]:
def knn(X, Y, X_new, k = 4):
    Y_pred = {}
    for i in X_new.index:
        distances = [np.sqrt(np.sum(X_new.loc[i] - X.loc[j]) ** 2) for j in X.index]
        indexes = np.argsort(distances)[:k]
        classes = [Y.iloc[l] for l in indexes]
        count = 0
        _class = None
        for j in classes:
            current_freq = classes.count(j)
            if current_freq > count:
                count = current_freq
                _class = j
        Y_pred[i] = _class

    return list(Y_pred.values())


7. Написать функцию для расчета метрик качества модели классификации. Функция должна возвращать значения оценок Precision, Recall, F1-score.

In [ ]:
def fscore(y, y_pred):
    TP = TN = FP = FN = 0
    for i in range(len(y)):
        if y_pred[i] == y[i]:
            if y_pred[i] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if y_pred[i] == 1:
                FP += 1
            else:
                FN += 1
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    f1_score = (2 * prec * rec) / (prec + rec)
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    return prec, rec, f1_score, accuracy

8. Выполнить предсказания классов для валидационной выборки для произвольного гиперпараметра k, рассчитать и вывести метрики Precision, Recall, F1-score и Accuracy.

In [ ]:
Y_val_pred = knn(X_train, Y_train, X_val, k=3)
prec, recall, f1_score, accuracy = fscore(np.array(Y_val), Y_val_pred)
print(f"prec: {round(prec, 3)}, recall: {round(recall, 3)}, f1 score: {round(f1_score, 3)}, accuracy: {round(accuracy, 3)}")


prec: 0.528, recall: 0.475, f1 score: 0.5, accuracy: 0.636


9. Задать массив или список значений гиперпараметра k (например, от 2 до 10 с шагом 2). Провести процедуру валидации, т.е. подбора оптимального значения k по критерию максимума точности. Для оценки точности использовать F1-score метрику. Выводить для каждого значения k полученное значение f1 score. В конце вывести лучший результат.

In [ ]:
K = [i for i in range(2,10)]
max_f1_score = 0
best_k = 0

for k in K:
    Y_val_pred = knn(X_train, Y_train, X_val, k)
    prec, recall, f1_score, accuracy = fscore(np.array(Y_val), Y_val_pred)
    print(f"k: {k}, f1_score: {f1_score}")
    if f1_score > max_f1_score:
        best_k = k
        max_f1_score = f1_score

print(f"Best k is: {best_k}")

k: 2, f1_score: 0.5932203389830508
k: 3, f1_score: 0.5
k: 4, f1_score: 0.5636363636363636
k: 5, f1_score: 0.5688073394495413
k: 6, f1_score: 0.5607476635514018
k: 7, f1_score: 0.6055045871559633
k: 8, f1_score: 0.6238532110091743
k: 9, f1_score: 0.6491228070175439
Best k is: 9


10. Рассчитать оценки точности (accuracy, precision, recal, F1-score) классификатора для тестовой выборки с оптимальным значением гиперпараметра k. Сделать вывод о качестве классификатора.

In [ ]:
Y_val_pred = knn(X_train, Y_train, X_val, k = best_k)
prec, recall, f1_score, accuracy = fscore(np.array(Y_val), Y_val_pred)
print(f"prec: {round(prec, 3)}, recall: {round(recall, 3)}, f1 score: {round(f1_score, 3)}, accuracy: {round(accuracy, 3)}")

prec: 0.673, recall: 0.627, f1 score: 0.649, accuracy: 0.74


11. Сравнить результаты с функцией KNeighborsClassifier из библиотеки sklearn. Если получился другой результат подумать в чем дело.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

param_grid = {'n_neighbors': range(2, 11)}  # Перебираем значения от 2 до 10
knn = KNeighborsClassifier(metric='euclidean')

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, Y_train)

best_k = grid_search.best_params_['n_neighbors']
print(f"Best k is: {best_k} with F1-score: {grid_search.best_score_}")

clf = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
clf.fit(X_train, Y_train)

Y_test_pred = clf.predict(X_test)

precision = precision_score(Y_test, Y_test_pred)
recall = recall_score(Y_test, Y_test_pred)
f1 = f1_score(Y_test, Y_test_pred)
accuracy = accuracy_score(Y_test, Y_test_pred)

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"F1 Score: {round(f1, 3)}")
print(f"Accuracy: {round(accuracy, 3)}")


Best k is: 7 with F1-score: 0.5695099818511797
Precision: 0.628
Recall: 0.54
F1 Score: 0.581
Accuracy: 0.747
